In [1]:
import pandas as pd
from neurocov.vis.utils import df_to_nifti

metric = "md"

data = pd.read_pickle("../data/data_combined.pickle").query("metric == @metric").drop("metric",axis=1)
parcels = pd.read_csv("../data/BNA_with_cerebellum.csv",index_col=0)
atlas_image_path = "../data/BN_Atlas_274_combined_1mm.nii.gz"

In [2]:
target = "temp_scaled"
stats = parcels.copy()
stats["r"] = data[parcels.Label.values].apply(lambda x: x.corr(data[target])).values

In [3]:
img = df_to_nifti(stats, atlas_image_path, "r", "Label")

Converting dataframe to nifti...
    Matching column: Label.
    Value column: r.


274it [00:02, 91.75it/s]


In [5]:
img.to_filename(f"../results/{metric}-{target}_correlations.nii.gz")